In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import re
from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

### 도구 호출 (Tool Calling)
> 웹 검색 기능을 AI모델에 통합해주는 웹 검색 API

- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음

랭체인 내장 도구 : Tavily

In [6]:
# poetry add langchain-tavily
# 최신 langchain 0.3.25 최신버전 기준  
# langchain-tavily 포함된 클래스를 사용

from langchain_tavily.tavily_search import TavilySearch

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearch(max_results=3)

# 웹 검색 실행
search_results = web_search.invoke(query)
print(type(search_results))
#pprint(search_results)

# 검색 결과 출력
for result in search_results['results']:
    print(type(result))
    pprint(result)  
    print("-" * 100)  

<class 'dict'>
<class 'dict'>
{'content': 'Published Time: 2022-01-27T11:22:15+09:00 스테이크와 어울리는 최고의 와인: 무엇을 '
            '고를 것인가? 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? Tags: 마시자매거진, 소고기와인, '
            '스테이크와인, 와인추천, 와인페어링 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 2018년 디캔터 월드 '
            '와인 어워드(Decanter World Wine Awards)에서 아르헨티나 지역 의장이었던 남미 와인 전문가이자 '
            '저널리스트인 파트리시오 타피아(Patricio Tapia)는 ‘나는 오크가 적고 신선한 과일과 더 좋은 산도를 가진 '
            '‘뉴 웨이브(new wave)’ 말벡을 선택하는 경향이 있다.’라고 말한다. ‘와인과 소고기를 페어링하는 가장 쉬운 '
            '방법은 소고기와 와인의 풍미 강도를 일치시키는 방법에 대해 생각하는 것이다.’라고 Hawksmoor 스테이크하우스 '
            '레스토랑의 와인 디렉터인 마크 퀵(Mark Quick)이 와인과 쇠고기의 페어링에 관한 심층 기사에서 언급했다. '
            'Tags: 마시자매거진 소고기와인 스테이크와인 와인추천 와인페어링 Bora Kim Previous Article '
            'Next Article Your email address will not be published. 대표자 : 방문송 '
            '사업자등록번호 : 325-87-00031 발행인 / 편집인 : 방문송',
 'raw_content': None,
 'score': 0.8668671,
 'title': '스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? - 마시자 매거진',
 'url': 'https://mashija.com/%E

In [7]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_tavily.tavily_search.TavilySearch'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. It not '
 'only retrieves URLs and snippets, but offers advanced search depths, domain '
 'management, time range filters, and image search, this tool delivers '
 'real-time, accurate, and citation-backed results.Input should be a search '
 'query.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input for [TavilySearch]',
 'properties': {'exclude_domains': {'anyOf': [{'items': {'type': 'string'},
                                               'type': 'array'},
        

### 도구 호출
- bind_tools로 LLM에 직접 바인딩

In [8]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(model="gpt-4o-mini")

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])
print(type(llm_with_tools))

<class 'langchain_core.runnables.base.RunnableBinding'>


In [9]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)
print(type(ai_msg))

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

<class 'langchain_core.messages.ai.AIMessage'>
AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 770, 'total_tokens': 781, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BmuAyMizNcDHk9j1zZZsPSYnjHqZp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--fa42ca2f-12b5-4b79-a342-419e1f3e3e7b-0', usage_metadata={'input_tokens': 770, 'output_tokens': 11, 'total_tokens': 781, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
----------------------------------------------------------------------------------------------------
'안녕하세요! 어떻게 도와드릴까요?'
---------

In [10]:
from pprint import pprint

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# AIMessage의 속성 확인
#pprint(dir(ai_msg))

In [11]:
pprint(ai_msg)
print("#" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("*" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0Oytr1cg4dIDFDbOZmpcAm3', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 779, 'total_tokens': 804, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BmuAzZtrdoCf3lnl0Xm0icgxgX9IH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f5d40a9e-3566-42a8-9e9e-86eabf3934e1-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_t0Oytr1cg4dIDFDbOZmpcAm3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 779, 'output_tokens': 25, 'total_tokens': 804,

### 도구 실행하기

#### 1. 직접 도구 호출 처리한 결과로 ToolMessage 생성
- 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
- 'args' 속성을 사용하여 TavilySearch 객체의 invoke() 호출하여 결과를 얻는다.

In [12]:
tool_call = ai_msg.tool_calls[0]

tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)

pprint(tool_output)

tavily_search 호출 결과:
----------------------------------------------------------------------------------------------------
{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '스테이크와 어울리는 와인 추천',
 'response_time': 0.92,
 'results': [{'content': 'Published Time: 2022-01-27T11:22:15+09:00 스테이크와 어울리는 '
                         '최고의 와인: 무엇을 고를 것인가? 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? '
                         'Tags: 마시자매거진, 소고기와인, 스테이크와인, 와인추천, 와인페어링 스테이크와 어울리는 '
                         '최고의 와인: 무엇을 고를 것인가? 2018년 디캔터 월드 와인 어워드(Decanter '
                         'World Wine Awards)에서 아르헨티나 지역 의장이었던 남미 와인 전문가이자 '
                         '저널리스트인 파트리시오 타피아(Patricio Tapia)는 ‘나는 오크가 적고 신선한 과일과 '
                         '더 좋은 산도를 가진 ‘뉴 웨이브(new wave)’ 말벡을 선택하는 경향이 있다.’라고 '
                         '말한다. ‘와인과 소고기를 페어링하는 가장 쉬운 방법은 소고기와 와인의 풍미 강도를 일치시키는 '
                         '방법에 대해 생각하는 것이다.’라고 Hawksmoor 스테이크하우스 레스토랑의 와인 디렉터인 '
                         '마크 퀵(Mark Quick)이 와인과 쇠

#### (추천) 방법 2: 도구 직접 호출하여 바로 ToolMessage 객체 생성
- tool_call 변수는 AIMessage 객체 포함된 tool 을 호출한 결과 tool_calls의 첫번째 dict 객체
- TavilySearch 객체를 invoke(tool_call) 하면 결과객체의 타입이 ToolMessage 이다.
- 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
- 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

In [13]:
# tool_call  {'name': 'tavily_search_results_json', 'args': {'query': 'wine pairing with steak'}, 'id': 'call_LrHyxTadqHDjW7J6LOWEaoSi', 'type': 'tool_call'}
tool_message = web_search.invoke(tool_call)
print(type(tool_message))

# 특정 속성들만 확인
print("\n=== 주요 속성들 ===")
attributes = ['content', 'tool_call_id', 'name', 'type', 'additional_kwargs']
for attr in attributes:
    if hasattr(tool_message, attr):
        print(f"{attr}: {getattr(tool_message, attr)}")

print("-" * 100)
pprint(tool_message.content)

<class 'langchain_core.messages.tool.ToolMessage'>

=== 주요 속성들 ===
content: {"query": "스테이크와 어울리는 와인 추천", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/", "title": "스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? - 마시자 매거진", "content": "Published Time: 2022-01-27T11:22:15+09:00 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? Tags: 마시자매거진, 소고기와인, 스테이크와인, 와인추천, 와인페어링 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 2018년 디캔터 월드 와인 어워드(Decanter World Wine Awards)에서 아르헨티나 지역 의장이었던 남미 와인 전문가이자 저널리스트인 파트리시오 타피아(Patricio Tapia)는 ‘나는 오크가 적고 신선한 과일과 더 좋은 산도를 가진 ‘뉴 웨이브(new wave)’ 말벡을 선택하는 경향이 있다.’라고 말한다. ‘와인과 소고기를 페어링하는 가장 쉬운 방법은 소고기와 와인의 풍미 강도를 일치시키는 방법에 대해 생각하는 것이다.’라고 Hawksmoor 스테이크하우스 레스토랑의 와인 디렉터인 마크 퀵(Mark Quick)이 와인과 쇠고기의 페어링에 관한 심층 기사에서 언급했다. Tags: 마시자매거진 소고기와인 스

#### 도구 호출이 여러개인 경우 - batch 사용

In [14]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

[ToolMessage(content='{"query": "스테이크와 어울리는 와인 추천", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/", "title": "스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? - 마시자 매거진", "content": "Published Time: 2022-01-27T11:22:15+09:00 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? Tags: 마시자매거진, 소고기와인, 스테이크와인, 와인추천, 와인페어링 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 2018년 디캔터 월드 와인 어워드(Decanter World Wine Awards)에서 아르헨티나 지역 의장이었던 남미 와인 전문가이자 저널리스트인 파트리시오 타피아(Patricio Tapia)는 ‘나는 오크가 적고 신선한 과일과 더 좋은 산도를 가진 ‘뉴 웨이브(new wave)’ 말벡을 선택하는 경향이 있다.’라고 말한다. ‘와인과 소고기를 페어링하는 가장 쉬운 방법은 소고기와 와인의 풍미 강도를 일치시키는 방법에 대해 생각하는 것이다.’라고 Hawksmoor 스테이크하우스 레스토랑의 와인 디렉터인 마크 퀵(Mark Quick)이 와인과 쇠고기의 페어링에 관한 심층 기사에서 언급했다. Tags: 마시자매거진 소고기와인 스테이크와인 와인추천 와인페어링 Bora Kim Previous Article Next Articl

### ToolMessage를 LLM에 전달하여 답변 생성

In [15]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_community.tools import TavilySearchResults

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")
print(type(llm))
# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)

tavily_tool = TavilySearchResults(max_results=2)
print(type(tavily_tool))

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_tool])
print(type(llm_with_tools))
#<class 'langchain_core.runnables.base.RunnableBinding'>

# LLM 체인 생성
llm_chain = prompt | llm_with_tools
print(type(llm_chain))

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
<class 'langchain_core.runnables.base.RunnableBinding'>
<class 'langchain_core.runnables.base.RunnableSequence'>


C:\Users\user\AppData\Local\Temp\ipykernel_35528\2864227807.py:21: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=2)


#### Chain decorator

In [16]:
# 도구 실행 체인 정의
# @chain 은 일반 함수를 Runnable 객체로 만들어주는 데코레이터 
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print(type(ai_msg))
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = tavily_tool.batch(ai_msg.tool_calls, config=config)
    print(type(tool_msgs))
    print("tool_msgs: \n", tool_msgs)
    
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
query = "스테이크와 어울리는 와인을 추천해주세요."
response = web_search_chain.invoke(query)

# 응답 출력 
pprint(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_j4yRfFROP0A3LrFhUb77Vjei', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 109, 'total_tokens': 136, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BmuB7j4G51LcrDv4FbY9MaePHCLY0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--c74d9e38-2214-42ea-95e1-5307ace20c79-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_j4yRfFROP0A3LrFhUb77Vjei', 'type': 'tool_call'}] usage_meta

#### Tool decorator

In [17]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [18]:

from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)
print(type(wiki_search))

<class 'langchain_core.tools.structured.StructuredTool'>


In [20]:
# 위키 검색 실행
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(type(result))
    print(result)  
    print("-" * 100)  

<class 'langchain_core.documents.base.Document'>
page_content='오르조(이탈리아어: orzo) 또는 리소니(이탈리아어: risoni, 단수: risone 리소네[*])는 이탈리아의 파스타이다. "오르조"는 라틴어:hordeum에서 유래했으며 "보리"를 뜻한다. "리소니"는 "큰 쌀"이라는 뜻이다. 파스타의 일종으로 큰 쌀알의 모양을 하고 있으며 솔방울이나 잣보다는 좀 더 작다. 보통 라구 등 수프와 함께 먹는다. 원래는 보리로 만들었지만 요즘에는 박력분으로 만드는 것이 흔해졌다. 다른 이름으로는 kritharáki ("little barley") 혹은 manéstra(그리스 요리)로 부르며, lisān al-`uṣfūr ("명금의 혀")라고 아랍 요리에서 불린다. 오르조를 두고 이탈리아의 쌀이라고 부르기도 한다.
터키에서는 아르파 셰흐리예(arpa şehriye)로 불리며, 셰흐리예의 하나이다. 필라브를 만들거나 초르바(수프)에 넣어 먹는다.


== 같이 보기 ==
파스티나
셰흐리예


== 각주 ==' metadata={'title': '오르조', 'summary': '오르조(이탈리아어: orzo) 또는 리소니(이탈리아어: risoni, 단수: risone 리소네[*])는 이탈리아의 파스타이다. "오르조"는 라틴어:hordeum에서 유래했으며 "보리"를 뜻한다. "리소니"는 "큰 쌀"이라는 뜻이다. 파스타의 일종으로 큰 쌀알의 모양을 하고 있으며 솔방울이나 잣보다는 좀 더 작다. 보통 라구 등 수프와 함께 먹는다. 원래는 보리로 만들었지만 요즘에는 박력분으로 만드는 것이 흔해졌다. 다른 이름으로는 kritharáki ("little barley") 혹은 manéstra(그리스 요리)로 부르며, lisān al-`uṣfūr ("명금의 혀")라고 아랍 요리에서 불린다. 오르조를 두고 이탈리아의 쌀이라고 부르기도 한다.\n터키에서는 아르파 셰흐리예(arpa şehriye)로 불리며, 셰흐리예의 하나이다. 필라브를 

In [21]:

# LLM에 도구를 바인딩 (2개의 도구 바인딩)
print(type(tavily_search_func)) # @tool 선언
print(type(wiki_search)) # as_tool() 함수사용

llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

<class 'langchain_core.tools.structured.StructuredTool'>
<class 'langchain_core.tools.structured.StructuredTool'>
AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wneOkyjmFwNVXCfZtSuaPec9', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집 추천"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_WqRlnKv2wVBVaEdply9VsEEB', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 176, 'total_tokens': 235, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BmuDebBVQFraNrHGUD4GR0Q2BCMBd', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--f05d

### LCEL을 통해 위키피디아 문서를 검색하고 내용을 요약

In [22]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"파스타의 유래"})
pprint(summarized_text)

('오르조(또는 리소니)는 이탈리아의 파스타로, 보리에서 유래한 이름을 가지고 있으며 큰 쌀알 모양이다. 주로 수프와 함께 먹으며, 현재는 '
 '박력분으로 만들어진다. 터키에서는 아르파 셰흐리예로 불리며, 필라브나 수프에 사용된다. \n'
 '\n'
 '이탈리아 요리는 기원전 4세기부터 발전해온 남유럽의 요리로, 지역마다 특색이 다르다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 '
 '주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다. 요리의 역사적 '
 '발전은 고대 그리스와 로마, 중세 아랍 문화의 영향을 받았으며, 다양한 요리책이 출판되었다.')


In [23]:
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

In [24]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3c60OCqnQtqzLvkLAlKbv1De', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_5O0QlpoLq7PF0KlGGGSVJxSS', 'function': {'arguments': '{"query": "파스타"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 152, 'total_tokens': 207, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BmuLRj77OtnTe4p6nbQKrLqmk0Bhr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--92ce5219-661b-425e-84b8-906b7e6ae16d-0', tool_calls=[{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'id':

In [25]:
# 도구 실행 
tool_message = wiki_summary.invoke(ai_msg.tool_calls[1])
print(type(tool_message))
print(tool_message)
print("-" * 100)
pprint(tool_message.content)

<class 'langchain_core.messages.tool.ToolMessage'>
content='파스타는 이탈리아의 전통 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 주식으로 여겨지며, 다양한 형태와 종류가 존재한다. 역사적으로 파스타는 고대 그리스와 로마에서 기원하며, 13세기부터 구체적인 기록이 나타난다. 건파스타와 생파스타로 나뉘며, 각각의 제조 방식과 재료가 다르다. 투움바 파스타는 미국의 아웃백 스테이크하우스에서 유래된 요리로, 매콤한 크림 소스를 사용하며 한국에서 인기를 끌고 있다.' name='wiki_summary' tool_call_id='call_5O0QlpoLq7PF0KlGGGSVJxSS'
----------------------------------------------------------------------------------------------------
('파스타는 이탈리아의 전통 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 주식으로 '
 '여겨지며, 다양한 형태와 종류가 존재한다. 역사적으로 파스타는 고대 그리스와 로마에서 기원하며, 13세기부터 구체적인 기록이 나타난다. '
 '건파스타와 생파스타로 나뉘며, 각각의 제조 방식과 재료가 다르다. 투움바 파스타는 미국의 아웃백 스테이크하우스에서 유래된 요리로, 매콤한 '
 '크림 소스를 사용하며 한국에서 인기를 끌고 있다.')


In [26]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_80X8pSFAa6Nw3EiIcQjZGmHs', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 120, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BmuNK0GcGFHAsXRKYarUUxjapEMPa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--ee2808ee-8fd1-425f-8a5b-1bc3055d421c-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_80X8pSFAa6Nw3EiIcQjZGmHs', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 19, 'total_tokens': 139, 'input_token_details': {